In [21]:
import time
from cryptography.fernet import Fernet
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.fernet import Fernet
import pandas as pd
import datetime
import re
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
import json
import warnings

warnings.filterwarnings("ignore")

In [22]:
# parameters for web scraping
today = datetime.date.today()
ano_atual, mes_atual, dia_atual = today.year, today.month, today.day
agora = datetime.datetime.now()

meses = {"jan": 1, "fev": 2, "mar": 3, "abr": 4, "mai": 5, "jun": 6, "jul": 7, "ago": 8, "set" :9, "out": 10, "nov": 11, "dez" :12}
meses_nome = {1: "janeiro", 2: "fevereiro", 3: "março", 4: "abril", 5: "maio", 6: "junho", 7: "julho", 8: "agosto", 9: "setembro", 10: "outubro", 11: "novembro", 12: "dezembro"}

colunas = ["updated_on", "data", "tipo", "valor"]
df_completo = pd.DataFrame(columns= colunas)

rede_url = 'https://meu.userede.com.br/relatorio/recebimentos'

# credentials
enc_google_creds_path = 'Credentials/enc_google_credentials.enc'
enc_credentials_path = 'Credentials/enc_credentials.enc'
enc_symmetric_key_path = 'Credentials/enc_symmetric_key.enc'
public_key_path = 'Credentials/public_key.pem'
private_key_path = 'Credentials/private_key.pem'

# parameter para google sheets
gsheet_name = 'https://docs.google.com/spreadsheets/d/1QWh8OOoPuIodDDNDSXF0GSwjWu3a45izAdRZFTPjj04'
scope = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]
sheet_name = "Rede_consolidado"

In [23]:
# Decrypt credentials 

# Load EC private key
with open(private_key_path, 'rb') as f:
    private_key = serialization.load_pem_private_key(f.read(), password=None)

# Load peer public key
with open(public_key_path, 'rb') as f:
    public_key = serialization.load_pem_public_key(f.read())

# Derive shared secret and AES key
shared_key = private_key.exchange(ec.ECDH(), public_key)
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data'
).derive(shared_key)

# Load and decrypt the symmetric key
with open(enc_symmetric_key_path, 'rb') as f:
    data = f.read()
nonce, enc_symmetric_key = data[:12], data[12:]
aesgcm = AESGCM(derived_key)
symmetric_key = aesgcm.decrypt(nonce, enc_symmetric_key, None)

fernet = Fernet(symmetric_key)

# Load encrypted credentials
with open(enc_credentials_path, 'rb') as f:
    enc_username, enc_password = f.read().split(b'\n')

username = fernet.decrypt(enc_username).decode()
password = fernet.decrypt(enc_password).decode()

In [24]:
# Set up Selenium WebDriver

options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Run in headless mode for efficiency

# Path to chromedriver
driver = webdriver.Chrome(options=options)
driver.maximize_window()

try:
    driver.get(rede_url)
    time.sleep(10)  

    # Locate username and password fields by their type attribute
    driver.find_element(By.XPATH,'//*[@id="user"]').send_keys(username)
    driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(password)
    driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(Keys.RETURN)
    time.sleep(3) 

    # uma possibilidade é apertar a tecla esc

    driver.get(rede_url)
    time.sleep(3)  
    driver.execute_script("document.querySelectorAll('.modal, .popup, .overlay').forEach(e => e.remove());")
    
    time.sleep(2)  
    driver.execute_script("document.body.style.zoom='80%'")

    print('Login attempted. Check for successful login or handle 2FA if required.')
except:
    print('An error occurred during login. Check your credentials or the website structure.')

Login attempted. Check for successful login or handle 2FA if required.


In [25]:
# TODO: mudar seleção de datas para pegar o mês atual e o próximo mês, e selecionar os dias entre 7 dias atrás e o final do próximo mês


# driver.find_element(By.XPATH, '//*[@id="date-filter"]/div[1]/div[1]/div').click()
# time.sleep(2)
# elemento = driver.find_element(By.XPATH, '//*[@id="date-filter"]/div[1]/div[3]')

# elemento_2 = elemento.find_elements(By.CLASS_NAME, 'layout horizontal')
# elemento_2
# # Click right arrow to go to next month
# right_arrow = driver.find_element(By.XPATH, '//paper-icon-button[@icon="av:chevron-right"]')
# right_arrow.click()
# time.sleep(1)

# elemento
# # Select dates between 7 days ago and the end of the next month in a calendar possibly inside an iframe

# # Calculate date range
# start_date = today - datetime.timedelta(days=7)
# next_month = (today.replace(day=1) + datetime.timedelta(days=32)).replace(day=1)
# end_date = (next_month + datetime.timedelta(days=calendar.monthrange(next_month.year, next_month.month)[1] - 1))


# # Switch to iframe if needed
# # try:
# iframe = driver.find_element(By.TAG_NAME, "iframe")
# driver.switch_to.frame(iframe)
# # except Exception:
# #     pass  # No iframe found, continue

# iframe

# # Open the date picker
# driver.find_element(By.XPATH, '//*[@id="date-filter"]/div[1]/div[1]/div').click()
# time.sleep(1)

# # Click right arrow to go to next month
# right_arrow = driver.find_element(By.XPATH, '//paper-icon-button[@icon="av:chevron-right"]')
# right_arrow.click()
# time.sleep(1)

# # Select dates from 7 days ago to end of next month
# current = start_date
# while current <= end_date:
#     try:
#         # Format day for selector (remove leading zero if present)
#         day_str = str(current.day)
#         # Try to find the day button in the calendar
#         day_elem = driver.find_element(By.XPATH, f'//div[contains(@class,"calendar") and text()="{day_str}"]')
#         day_elem.click()
#         time.sleep(0.1)
#     except Exception:
#         pass  # Date not found in current view, skip
#     current += datetime.timedelta(days=1)

# # Switch back to default content if iframe was used
# try:
#     driver.switch_to.default_content()
# except Exception:
#     pass

In [26]:
elementos = driver.find_elements(By.CLASS_NAME, "report-receipts-daily-content")

for elemento in elementos:
    data = elemento.find_element(By.CLASS_NAME, "date-title").text
    match = re.search(r'(\d{1,2})/([a-z]{3})', data, re.IGNORECASE)
    if match:
        dia = int(match.group(1))
        mes = meses[match.group(2).lower()]
        if mes == 1 and mes_atual == 12:
            ano = ano_atual + 1
        elif mes == 12 and mes_atual == 1:
            ano = ano_atual - 1
        else:
            ano = ano_atual       
        data = datetime.date(ano, mes, dia)
    valor = elemento.find_elements(By.CLASS_NAME, 'rede-big-number-description')[1].text 
    valor = re.sub(r'[^\d,]', '', valor).replace(',', '.')
    valor = float(valor) if valor else 0.0
    valor = int(valor * 100)
    df_temp = pd.DataFrame([[agora, data, "realizado", valor]], columns=colunas)
    df_completo = pd.concat([df_completo, df_temp], ignore_index=True)

del df_temp

In [27]:
driver.find_element(By.XPATH, '//*[@id="recebiveis_content"]').click()
time.sleep(10)

elementos = driver.find_elements(By.CLASS_NAME, "receivables-daily-content-report-receipts.ng-star-inserted")

for elemento in elementos:
    data = elemento.find_element(By.CLASS_NAME, "date-title").text
    match = re.search(r'(\d{1,2})/([a-z]{3})', data, re.IGNORECASE)
    if match:
        dia = int(match.group(1))
        mes = meses[match.group(2).lower()]
        if mes == 1 and mes_atual == 12:
            ano = ano_atual + 1
        elif mes == 12 and mes_atual == 1:
            ano = ano_atual - 1
        else:
            ano = ano_atual       
        data = datetime.date(ano, mes, dia)
    valor = elemento.find_elements(By.CLASS_NAME, 'rede-big-number-description')[1].text 
    valor = re.sub(r'[^\d,]', '', valor).replace(',', '.')
    valor = float(valor) if valor else 0.0
    valor = int(valor * 100)
    df_temp = pd.DataFrame([[agora, data, "previsto", valor]], columns=colunas)
    df_completo = pd.concat([df_completo, df_temp], ignore_index=True)

del df_temp
driver.quit()

df_completo = df_completo.sort_values(by=["data"], ascending=[True])
df_completo = df_completo.reset_index(drop=True)

df_completo

,updated_on,data,tipo,valor
0,2025-05-29 17:01:32.747908,2025-05-23,realizado,10616406
1,2025-05-29 17:01:32.747908,2025-05-26,realizado,42408150
2,2025-05-29 17:01:32.747908,2025-05-27,realizado,44674621
3,2025-05-29 17:01:32.747908,2025-05-28,realizado,8752012
4,2025-05-29 17:01:32.747908,2025-05-29,realizado,8102525
5,2025-05-29 17:01:32.747908,2025-05-30,previsto,9253818
6,2025-05-29 17:01:32.747908,2025-06-02,previsto,44300175
7,2025-05-29 17:01:32.747908,2025-06-03,previsto,46928141
8,2025-05-29 17:01:32.747908,2025-06-04,previsto,20041339
9,2025-05-29 17:01:32.747908,2025-06-05,previsto,15645682


In [28]:
# Load encrypted credentials
with open(enc_google_creds_path, 'rb') as f:
    enc_credentials = f.read().strip()
credentials = fernet.decrypt(enc_credentials).decode()

# Authenticate and connect to Google Sheets
creds = Credentials.from_service_account_info(json.loads(credentials), scopes=scope)
gc = gspread.authorize(creds)

In [29]:
# Open the Google Sheet
sh = gc.open_by_url(gsheet_name) if gsheet_name.startswith('https://') else gc.open(gsheet_name)
worksheet = sh.worksheet(sheet_name)  

df_total = pd.DataFrame(worksheet.get_all_records())
min_data = df_completo['data'].min()
df_total['data'] = pd.to_datetime(df_total['data'], errors='coerce').dt.date 
df_total_filtered = df_total[df_total['data'] >= min_data].copy()
df_total_inicial = df_total[df_total['data'] < min_data].copy()

In [30]:
# Ensure 'updated_on' is datetime for all dataframes before deduplication
df_completo['updated_on'] = pd.to_datetime(df_completo['updated_on'], errors='coerce')
df_total['updated_on'] = pd.to_datetime(df_total['updated_on'], errors='coerce')
df_total_filtered['updated_on'] = pd.to_datetime(df_total_filtered['updated_on'], errors='coerce')

# Re-concatenate and deduplicate as per the rules
df_concat = pd.concat([df_total_filtered, df_completo], axis=0)
df_concat = df_concat.sort_values(by=["data", "tipo", "updated_on"], ascending=[True, True, False])

def select_row(group):
    # Always prefer 'realizado' if present, and pick the latest 'updated_on' among them
    if (group['tipo'] == 'realizado').any():
        sub = group[group['tipo'] == 'realizado']
        return sub.loc[sub['updated_on'].idxmax()]
    # Otherwise, pick the row with the latest 'updated_on'
    return group.loc[group['updated_on'].idxmax()]

# Apply the rule and ensure only one row per date
result = df_concat.groupby('data', group_keys=False).apply(select_row)
df_final = result.reset_index(drop=True)

# Remove any accidental duplicate 'data' rows (shouldn't be needed, but for safety)
df_final = df_final.drop_duplicates(subset=['data'], keep='first')

df_final = pd.concat([df_total_inicial, df_final], ignore_index=True)
df_final = df_final.sort_values(by=["data"], ascending=[True]).reset_index(drop=True)

# 4. Delete from the sheet the rows with 'data' >= min_data
if len(worksheet.get_all_values()) > 1: 
    linha_inicial_delete = 2
    linha_final_delete = len(worksheet.get_all_values())
    worksheet.delete_rows(linha_inicial_delete, linha_final_delete)

# 5. Append the new dataframe to the end of the sheet
from gspread_dataframe import set_with_dataframe
last_row = len(worksheet.get_all_values())
set_with_dataframe(worksheet, df_final, row=last_row+1, include_column_header=False)

df_final

,updated_on,data,tipo,valor
0,2025-05-29 17:01:32.747908,2025-05-23,realizado,10616406
1,2025-05-29 17:01:32.747908,2025-05-26,realizado,42408150
2,2025-05-29 17:01:32.747908,2025-05-27,realizado,44674621
3,2025-05-29 17:01:32.747908,2025-05-28,realizado,8752012
4,2025-05-29 17:01:32.747908,2025-05-29,realizado,8102525
5,2025-05-29 17:01:32.747908,2025-05-30,previsto,9253818
6,2025-05-29 17:01:32.747908,2025-06-02,previsto,44300175
7,2025-05-29 17:01:32.747908,2025-06-03,previsto,46928141
8,2025-05-29 17:01:32.747908,2025-06-04,previsto,20041339
9,2025-05-29 17:01:32.747908,2025-06-05,previsto,15645682
